In [4]:
# Imports

import joblib
import sklearn
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

In [5]:
%reload_ext watermark
%watermark -a "Marcelo Medeiros | Cientista de Dados"

Author: Marcelo Medeiros | Cientista de Dados



### Carregando os Dados

In [6]:
# Data
df = pd.read_csv("dataset.csv")

In [7]:
df.shape

(10000, 6)

In [8]:
df.head()

,vibracao,temperatura,pressao,umidade,horas_trabalho,manutencao_necessaria
0,0.250951,92.419225,100.311847,67.596275,7499,0
1,0.895355,69.132552,96.137413,70.454398,600,0
2,0.564789,66.456903,93.642299,31.822434,6919,0
3,0.853165,81.967579,101.924996,46.543886,4032,1
4,2.143944,60.097525,97.527537,50.129838,8036,0


In [9]:
# Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   vibracao               10000 non-null  float64
 1   temperatura            10000 non-null  float64
 2   pressao                10000 non-null  float64
 3   umidade                10000 non-null  float64
 4   horas_trabalho         10000 non-null  int64  
 5   manutencao_necessaria  10000 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 468.9 KB


In [10]:
# Prorpoção de Classe
df.manutencao_necessaria.value_counts()

manutencao_necessaria
1    5517
0    4483
Name: count, dtype: int64

- 1 é a classe positiva (foi necessário manutenção na máquina)
- 0 é a classe negativa (não foi necessário manutenção na máquina)

### Preparação dos Dados

In [11]:
# Separando variáveis explicativas e a alvo
X = df.drop("manutencao_necessaria", axis=1)
y = df["manutencao_necessaria"]

In [12]:
# Dividir os dados em conjunto de treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#### Opção 1 - Ajuste de Pesos no Modelo (Sem Reamostragem)

In [13]:
scaler_v1 = StandardScaler()
X_train_scaled = scaler_v1.fit_transform(X_train)
X_test_scaled = scaler_v1.transform(X_test)

In [14]:
# Instanciar o modelo com ajuste de pesos
modelo_v1 = RandomForestClassifier(n_estimators=100,
                                   random_state=42,
                                   class_weight='balanced')

In [15]:
%%time
modelo_v1.fit(X_train_scaled, y_train)

CPU times: total: 2.69 s
Wall time: 3.78 s


RandomForestClassifier(class_weight='balanced', random_state=42)

In [16]:
# Fazer previsões no conjunto de teste
y_pred = modelo_v1.predict(X_test_scaled)
y_pred_proba = modelo_v1.predict_proba(X_test_scaled)[:, 1]

In [17]:
# Avaliação do Modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 98.70%
AUC-ROC: 98.78%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99       876
           1       0.99      0.98      0.99      1124

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



In [19]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v1.pkl'
scaler_file = 'padronizadores/scaler_v1.pkl'

joblib.dump(modelo_v1, model_file)
joblib.dump(scaler_v1, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v1.pkl
Scaler salvo em padronizadores/scaler_v1.pkl


### Opção 2 - Subamostragem da Classe Majoritária (Undersampling)

In [18]:
# Proporção de classe
df.manutencao_necessaria.value_counts()

manutencao_necessaria
1    5517
0    4483
Name: count, dtype: int64

In [20]:
# Concatenar X_train e y_train para facilitar a reamostragem
train_data = pd.concat([X_train, y_train], axis = 1)

In [21]:
# Separar a classe majoritária e minoritária do conjunto de treino
df_majoritaria = train_data[train_data.manutencao_necessaria == 1]
df_minoritaria = train_data[train_data.manutencao_necessaria == 0]

In [22]:
len(df_majoritaria)

4393

In [23]:
len(df_minoritaria)

3607

In [24]:
# Aplicar subamostragem da classe majoritária no conjunto de treino
df_majoritaria_subamostrada = resample(df_majoritaria,
                                       replace = False,    
                                       n_samples = len(df_minoritaria),  # Igualar o número da classe minoritária
                                       random_state = 42)

In [25]:
# Combinar as classes minoritária e majoritária subamostrada
train_data_balanceado = pd.concat([df_majoritaria_subamostrada, df_minoritaria])

In [26]:
# Separar novamente em X_train e y_train balanceados
X_train_balanceado = train_data_balanceado.drop('manutencao_necessaria', axis = 1)
y_train_balanceado = train_data_balanceado['manutencao_necessaria']

In [27]:
# Verificar o balanceamento no conjunto de treino
print(y_train_balanceado.value_counts())

manutencao_necessaria
1    3607
0    3607
Name: count, dtype: int64


In [28]:
# Padronizar os dados
scaler_v2 = StandardScaler()
X_train_scaled = scaler_v2.fit_transform(X_train_balanceado)
X_test_scaled = scaler_v2.transform(X_test)

In [29]:
# Instanciar e treinar o modelo
modelo_v2 = RandomForestClassifier(n_estimators = 100, random_state = 42)

In [30]:
%%time
modelo_v2.fit(X_train_scaled, y_train_balanceado)

CPU times: total: 2.3 s
Wall time: 3.6 s


RandomForestClassifier(random_state=42)

In [31]:
# Avaliar o modelo no conjunto de teste 
y_pred = modelo_v2.predict(X_test_scaled)
y_pred_proba = modelo_v2.predict_proba(X_test_scaled)[:, 1]

In [32]:
# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 98.20%
AUC-ROC: 98.28%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98       876
           1       0.99      0.98      0.98      1124

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000



In [33]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v2.pkl'
scaler_file = 'padronizadores/scaler_v2.pkl'

joblib.dump(modelo_v2, model_file)
joblib.dump(scaler_v2, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v2.pkl
Scaler salvo em padronizadores/scaler_v2.pkl


### Opção 3 - Superamostragem da Classe Minoritária (Oversampling)

In [34]:
# Concatenar X_train e y_train para facilitar a reamostragem
train_data = pd.concat([X_train, y_train], axis = 1)

In [35]:
# Separar a classe majoritária e minoritária do conjunto de treino
df_majoritaria = train_data[train_data.manutencao_necessaria == 1]
df_minoritaria = train_data[train_data.manutencao_necessaria == 0]

In [36]:
len(df_majoritaria)

4393

In [37]:
len(df_minoritaria)

3607

In [38]:
# Superamostragem da classe minoritária no conjunto de treino
df_minoritaria_superamostrada = resample(df_minoritaria,
                                         replace = True,     
                                         n_samples = len(df_majoritaria), # Igualar ao número da classe majoritária
                                         random_state = 42)

In [39]:
# Combinar as classes majoritária e minoritária superamostrada
train_data_balanceado = pd.concat([df_majoritaria, df_minoritaria_superamostrada])

In [40]:
# Separar novamente em X_train e y_train balanceados
X_train_balanceado = train_data_balanceado.drop('manutencao_necessaria', axis = 1)
y_train_balanceado = train_data_balanceado['manutencao_necessaria']

In [41]:
# Verificar o balanceamento no conjunto de treino
print(y_train_balanceado.value_counts())

manutencao_necessaria
1    4393
0    4393
Name: count, dtype: int64


In [42]:
# Padronizar os dados
scaler_v3 = StandardScaler()
X_train_scaled = scaler_v3.fit_transform(X_train_balanceado)
X_test_scaled = scaler_v3.transform(X_test)

In [43]:
# Criar o modelo
modelo_v3 = RandomForestClassifier(n_estimators = 100, random_state = 42)

In [44]:
%%time
modelo_v3.fit(X_train_scaled, y_train_balanceado)

CPU times: total: 1.91 s
Wall time: 3.43 s


RandomForestClassifier(random_state=42)

In [45]:
# Avaliar o modelo no conjunto de teste 
y_pred = modelo_v3.predict(X_test_scaled)
y_pred_proba = modelo_v3.predict_proba(X_test_scaled)[:, 1]

In [46]:
# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 98.25%
AUC-ROC: 98.96%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       876
           1       0.98      0.98      0.98      1124

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000



In [47]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v3.pkl'
scaler_file = 'padronizadores/scaler_v3.pkl'

joblib.dump(modelo_v3, model_file)
joblib.dump(scaler_v3, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v3.pkl
Scaler salvo em padronizadores/scaler_v3.pkl


### Opção 4 - Balanceamento Automático com SMOTE

In [48]:
# Padronização
scaler_v4 = StandardScaler()
X_train_scaled = scaler_v4.fit_transform(X_train)
X_test_scaled = scaler_v4.transform(X_test)

In [49]:
# # Criando o SMOTE
smote = SMOTE(random_state=42)

In [50]:
# Treinar e aplicar SMOTE no conjunto de treino
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

In [51]:
len(X_train_smote)

8786

In [52]:
# Criar o modelo
modelo_v4 = RandomForestClassifier(n_estimators = 100, random_state = 42)

In [53]:
%%time
modelo_v4.fit(X_train_smote, y_train_smote)

CPU times: total: 1.58 s
Wall time: 4.07 s


RandomForestClassifier(random_state=42)

In [54]:
# Avaliar o modelo no conjunto de teste 
y_pred = modelo_v4.predict(X_test_scaled)
y_pred_proba = modelo_v4.predict_proba(X_test_scaled)[:, 1]

In [55]:
# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 98.85%
AUC-ROC: 99.03%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99       876
           1       0.99      0.98      0.99      1124

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



In [56]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v4.pkl'
scaler_file = 'padronizadores/scaler_v4.pkl'

joblib.dump(modelo_v4, model_file)
joblib.dump(scaler_v4, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v4.pkl
Scaler salvo em padronizadores/scaler_v4.pkl


### Opção 5 - Balanceamento Automático com SMOTE e Mudança de Algoritmo

- SMOTE Primeiro e Padronização Depois!

In [57]:
%%time

# Criar o SMOTE
smote = SMOTE(random_state = 42)

# Treinar e aplicar SMOTE no conjunto de treino
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Padronização
scaler_v5 = StandardScaler()
X_train_scaled = scaler_v5.fit_transform(X_train_smote)
X_test_scaled = scaler_v5.transform(X_test)

# Criar o modelo
modelo_v5 = lgb.LGBMClassifier(random_state = 42)

# Treinar o modelo com o conjunto de treino balanceado via SMOTE
modelo_v5.fit(X_train_scaled, y_train_smote)

# Avaliar o modelo no conjunto de teste 
y_pred = modelo_v4.predict(X_test_scaled)
y_pred_proba = modelo_v4.predict_proba(X_test_scaled)[:, 1]

# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

[LightGBM] [Info] Number of positive: 4393, number of negative: 4393
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000945 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 8786, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000

Acurácia: 96.30%
AUC-ROC: 98.53%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.94      0.98      0.96       876
           1       0.99      0.95      0.97      1124

    accuracy                           0.96      2000
   macro avg       0.96      0.97      0.96      2000
weighted avg       0.96      0.96      0.96      2000

CPU times: total: 391 ms
Wall time: 352 ms


- Padronização Primeiro e SMOTE Depois!

In [58]:
%%time

# Padronizar as variáveis
scaler_v5 = StandardScaler()
X_train_scaled = scaler_v5.fit_transform(X_train)
X_test_scaled = scaler_v5.transform(X_test)

# Criar o SMOTE
smote = SMOTE(random_state = 42)

# Aplicar SMOTE no conjunto de treino para lidar com o desbalanceamento
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Criar o modelo LightGBM
modelo_v5 = lgb.LGBMClassifier(random_state = 42)

# Treinar o modelo LightGBM
modelo_v5.fit(X_train_smote, y_train_smote)

# Avaliar o modelo no conjunto de teste original
y_pred = modelo_v5.predict(X_test_scaled)
y_pred_proba = modelo_v5.predict_proba(X_test_scaled)[:, 1]

# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nAcurácia: {accuracy * 100:.2f}%")
print(f"AUC-ROC: {roc_auc * 100:.2f}%")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

[LightGBM] [Info] Number of positive: 4393, number of negative: 4393
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000265 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 8786, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000

Acurácia: 92.40%
AUC-ROC: 97.47%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.86      0.98      0.92       876
           1       0.98      0.88      0.93      1124

    accuracy                           0.92      2000
   macro avg       0.92      0.93      0.92      2000
weighted avg       0.93      0.92      0.92      2000

CPU times: total: 219 ms
Wall time: 145 ms


In [59]:
# Salva modelo e padronizador em disco
model_file = 'modelos/modelo_v5.pkl'
scaler_file = 'padronizadores/scaler_v5.pkl'

joblib.dump(modelo_v5, model_file)
joblib.dump(scaler_v5, scaler_file)

print(f"Modelo salvo em {model_file}")
print(f"Scaler salvo em {scaler_file}")

Modelo salvo em modelos/modelo_v5.pkl
Scaler salvo em padronizadores/scaler_v5.pkl


**Seleção de Modelo**

**Versão 1 do Modelo:**

- Acurácia: 98.70%
- AUC-ROC: 98.78%

**Versão 2 do Modelo:**

- Acurácia: 98.20%
- AUC-ROC: 98.28%

**Versão 3 do Modelo:**

- Acurácia: 98.25%
- AUC-ROC: 98.96%

**Versão 4 do Modelo:**

- Acurácia: 98.85%
- AUC-ROC: 99.03%

**Versão 5 do Modelo:**

**SMOTE**

- Acurácia: 92.40%
- AUC-ROC: 97.47%

A versão 1 do modelo foi a que apresentou o maior equilíbrio entre:

**Capacidade de Generalização / Performance / Simplicidade / Interpretabilidade**

O desafio é encontrar o modelo que consiga equilibrar esses 4 elementos.

### Testando o Deploy do Modelo

In [61]:
# Função para recomendar manutenção baseada em novos dados de sensores IoT
def recomendacao_manutencao(novo_dado):
    
    # Definir os nomes das colunas conforme o scaler foi ajustado
    colunas = ['vibracao', 'temperatura', 'pressao', 'umidade', 'horas_trabalho']
    
    # Converter os novos dados para DataFrame com os nomes de colunas corretos
    novo_dado_df = pd.DataFrame([novo_dado], columns = colunas)
    
    # Aplicar o scaler aos novos dados
    novo_dado_scaled = scaler_v1.transform(novo_dado_df)
    
    # Fazer a previsão
    predicao = modelo_v1.predict(novo_dado_scaled)
    
    if predicao == 1:
        return "Recomendação: Realizar manutenção."
    else:
        return "Recomendação: Nenhuma manutenção necessária."

In [62]:
# Exemplo de novos dados de sensores IoT 
novos_dados_1 = [0.5, 80, 102, 45, 8000]
print(recomendacao_manutencao(novos_dados_1))

Recomendação: Realizar manutenção.


In [63]:
# Exemplo de novos dados de sensores IoT 
novos_dados_2 = [0.89, 92, 96, 70, 600]
print(recomendacao_manutencao(novos_dados_2))

Recomendação: Nenhuma manutenção necessária.


In [64]:
%watermark -a "Marcelo Medeiros | Cientista de Dados"

Author: Marcelo Medeiros | Cientista de Dados



In [65]:
%watermark -v -m

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.15.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [66]:
%watermark --iversions

lightgbm: 4.5.0
pandas  : 2.0.3
joblib  : 1.2.0
sklearn : 1.3.0
numpy   : 1.24.3



# FIM